---
title: Módulo 4
subtitle: Métodos de remuestreo
author:
  - name: Eloy Alvarado Narváez
    orcid: 0000-0001-7522-2327
    email: eloy.alvarado@usm.cl
    affiliations: Universidad Técnica Federico Santa María
  - name: Esteban Salgado Valenzuela
    orcid: 0000-0002-7799-0044
    affiliations: Universidad Técnica Federico Santa María
date: 14/12/2024
---

En este laboratorio exploraremos las técnicas de remuestreo tratadas en este capítulo. Ten en cuenta que algunos de los comandos utilizados aquí pueden tardar un tiempo en ejecutarse en tu computadora.

Nuevamente, comenzaremos colocando nuestros imports en este nivel superior.

In [78]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

#### Conjunto de Validación

En esta sección exploraremos el uso del enfoque de conjunto de validación para estimar las tasas de error en el conjunto de prueba derivadas de ajustar diversos modelos lineales al conjunto de datos `Auto`.

Utilizaremos la función `train_test_split()` para dividir los datos en conjuntos de entrenamiento y validación. Dado que contamos con 392 observaciones, las separaremos en dos conjuntos iguales de 196 registros cada uno, empleando el argumento `test_size=196`. En general, es una buena práctica establecer una semilla aleatoria al realizar operaciones de este tipo, que incluyen un elemento de aleatoriedad, con el fin de poder reproducir exactamente los resultados más adelante. Para ello, fijamos la semilla del generador aleatorio con el argumento `random_state=0`.

In [79]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)

Ahora podemos ajustar un modelo de regresión lineal utilizando únicamente las observaciones correspondientes al conjunto de entrenamiento `Auto_train`.

In [80]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

Ahora empleamos el método `predict()` de `results`, evaluándolo sobre la matriz de modelo generada con el conjunto de datos de validación. Además, calculamos el MSE (Error Cuadrático Medio) de validación de nuestro modelo.

In [81]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

Por lo tanto, nuestra estimación del MSE de validación para el ajuste de la regresión lineal es $23.62$.

Asimismo, podemos estimar el error de validación para regresiones polinómicas de mayor orden. Primero proporcionamos una función, `evalMSE()`, que recibe como entrada una cadena que describe el modelo, así como los conjuntos de entrenamiento y prueba, y devuelve el MSE en el conjunto de prueba.

In [82]:
def evalMSE(terms,
            response,
            train,
            test):

   mm = MS(terms)
   X_train = mm.fit_transform(train)
   y_train = train[response]

   X_test = mm.transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2)

Utilicemos esta función para estimar el MSE de validación empleando ajustes lineales, cuadráticos y cúbicos. Aquí recurrimos a la función `enumerate()`, la cual proporciona tanto los valores como los índices de los elementos mientras iteramos con un bucle for.

In [83]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

Estas tasas de error son $23.62$, $18.76$ y $18.80$, respectivamente. Si optamos por una división distinta entre entrenamiento y validación, es de esperar que obtengamos errores algo diferentes en el conjunto de validación.

In [84]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

Usando esta división de las observaciones entre el conjunto de entrenamiento y el de validación, observamos que las tasas de error en el conjunto de validación para los modelos con términos lineales, cuadráticos y cúbicos son $20.76$, $16.94$ y $16.97$, respectivamente.

Estos resultados coinciden con nuestros hallazgos previos: un modelo que predice `mpg` empleando una función cuadrática del `horsepower` supera a uno que se basa únicamente en una función lineal, y no existe evidencia de mejora al emplear una función cúbica del `horsepower`.

#### Validación Cruzada

En teoría, la estimación mediante validación cruzada puede calcularse para cualquier modelo lineal generalizado (GLM). Sin embargo, en la práctica, la forma más sencilla de realizar validación cruzada en `Python` es utilizando `sklearn`, que presenta una interfaz o API diferente a la de `statsmodels`, el paquete que hemos estado utilizando para ajustar GLMs.

Este es un problema que a menudo afrontan los científicos de datos: “Tengo una función que realiza la tarea $A$, y necesito alimentarla a otra que realiza la tarea $B$, de modo que pueda calcular $B(A(D))$, donde $D$ es mi conjunto de datos.” Cuando $A$ y $B$ no se comunican naturalmente entre sí, es necesario recurrir a un envoltorio (*wrapper*). En el paquete `ISLP` se proporciona la clase `sklearn_sm()`, que nos permite utilizar fácilmente las herramientas de validación cruzada de `sklearn` con modelos ajustados mediante `statsmodels`.

La clase `sklearn_sm()` tiene como primer argumento un modelo de `statsmodels`. Puede además recibir dos argumentos opcionales: `model_str`, para especificar una fórmula, y `model_args`, que debe ser un diccionario con argumentos adicionales utilizados al ajustar el modelo. Por ejemplo, para ajustar un modelo de regresión logística es necesario especificar el argumento `family`. Esto se hace pasando `model_args={'family':sm.families.Binomial()}`.

In [91]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

Los argumentos de la función `cross_validate()` son los siguientes: un objeto con los métodos `fit()`, `predict()` y `score()` adecuados, un arreglo de predictores `X` y una respuesta `Y`. Además, incluimos un argumento adicional `cv` en `cross_validate()`. Al especificar un entero $K$ da lugar a una validación cruzada de tipo $K$-fold. Aquí le hemos asignado un valor igual al número total de observaciones, lo que resulta en una validación cruzada *"leave-one-out"* (LOOCV). La función `cross_validate()` produce un diccionario con varios componentes; en este caso, simplemente nos interesa el valor de la métrica de prueba con validación cruzada (MSE), estimado en $24.23$.

Podemos repetir este procedimiento para ajustes polinómicos cada vez más complejos. Para automatizar el proceso, utilizamos nuevamente un bucle for que itera sobre polinomios de grado 1 a 5, calcula el error de validación cruzada asociado y lo almacena en el elemento correspondiente del vector `cv_error`. La variable `d` en el bucle for corresponde al grado del polinomio. Comenzamos inicializando el vector. Esta ejecución puede tardar un par de segundos.

In [10]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

Apreciamos una marcada disminución en el MSE estimado de prueba al pasar del ajuste lineal al cuadrático, pero no se observa una mejora clara al emplear polinomios de mayor grado.

Arriba presentamos el método `outer()` de la función `np.power()`. El método `outer()` se aplica a una operación que requiere dos argumentos, como `add()`, `min()` o `power()`. Este método recibe dos arreglos como argumentos y produce un arreglo más grande donde la operación se aplica a cada par de elementos de dichos arreglos.

En el ejemplo de validación cruzada anterior empleamos $K=n$, pero naturalmente también podemos utilizar $K<n$. El código es muy similar al anterior (y resulta significativamente más rápido). Aquí utilizaremos `KFold()` para particionar los datos en grupos aleatorios. Además, emplearemos `random_state` para establecer una semilla aleatoria e inicializaremos un vector `cv_error` donde almacenaremos los errores de validación cruzada correspondientes a los ajustes polinómicos de grado uno a cinco.

In [11]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # usar mismos conjuntos en cada grado
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720065])

Observemos que el tiempo de cómputo es mucho menor que el de LOOCV. (En principio, el tiempo de cómputo de LOOCV para un modelo lineal de mínimos cuadrados debería ser más rápido que para una validación cruzada de tipo $K$-fold, debido a la disponibilidad de la fórmula 

$$CV_{(n)} = \frac{1}{n}\sum\limits_{i=1}^n\left(\frac{y_i-\hat{y}_i}{1-h_i}\right)^2$$

 para LOOCV; sin embargo, la función genérica `cross_validate()` no hace uso de dicha fórmula.) Aun así, seguimos sin encontrar evidencia de que el uso de términos cúbicos o de mayor grado genere un error de prueba menor que el simple ajuste cuadrático.

La función `cross_validate()` es flexible y puede recibir distintos mecanismos de separación como argumento. Por ejemplo, se puede emplear la función `ShuffleSplit()` para implementar el enfoque de conjunto de validación con la misma facilidad que la $K$-fold CV.

In [12]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score']

array([23.61661707])

Se puede estimar la variabilidad en el error de prueba ejecutando el siguiente comando:

In [13]:
results['test_score'].mean(), results['test_score'].std()

(23.61661706966988, 0.0)

Tengamos en cuenta que esta desviación estándar no es una estimación válida de la variabilidad muestral de la media del error de prueba ni de los valores individuales, ya que las muestras de entrenamiento seleccionadas aleatoriamente se solapan y esto introduce correlaciones. Sin embargo, sí proporciona una idea de la variación de *Monte Carlo* que surge al escoger diferentes pliegues aleatorios.

#### El Bootstrap

Ilustraremos el uso de *bootstrap* con dos problemas.

##### Estimando la Precisión de una Estadística de Interés

Una de las grandes ventajas del enfoque bootstrap es que puede aplicarse en prácticamente cualquier situación, sin requerir complicados cálculos matemáticos. Aunque existen varias implementaciones de bootstrap en `Python`, su uso para estimar el error estándar es lo suficientemente sencillo como para que escribamos nuestra propia función cuando los datos se almacenan en un `DataFrame`.

Comenzaremos con un ejemplo simple. El conjunto de datos `Portfolio`, incluido en el paquete `ISLP`. El objetivo es estimar la varianza muestral del parámetro $\alpha$

$$\hat{\alpha} = \dfrac{\hat\sigma^2_Y - \hat\sigma_{XY}}{\hat\sigma^2_X+\hat\sigma^2_Y-2\hat\sigma_{XY}}$$

Crearemos una función `alpha_func()`, la cual toma como entrada un dataframe `D` que se asume contiene columnas `X` e `Y`, así como un vector `idx` que indica qué observaciones se utilizarán para estimar $\alpha$. La función devuelve la estimación de basada en las observaciones seleccionadas.

In [17]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

Portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


Esta función devuelve una estimación de $\alpha$ a partir de la fórmula de varianza mínima a las observaciones indexadas por el argumento `idx`. Por ejemplo, el siguiente comando estima utilizando las 100 observaciones completas.

In [16]:
alpha_func(Portfolio, range(100))

0.57583207459283

A continuación, seleccionamos aleatoriamente 100 observaciones de `range(100)`, con reemplazo. Esto equivale a construir un nuevo conjunto de datos bootstrap y a recomputar $\hat\alpha$ basado en el nuevo conjunto de datos.

In [19]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619004

Este proceso puede generalizarse para crear una función sencilla, `boot_SE()`, que calcule el error estándar mediante bootstrap para funciones arbitrarias que reciban únicamente un dataframe como argumento.

In [70]:
def boot_SE(func,D,n=None,B=1000,seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / (B-1) - B/(B-1)*(first_ / B)**2)

Usemos esta función para evaluar la precisión de nuestra estimación de $\alpha$ usando $B=1000$ repeticiones de bootstrap.

In [71]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09122739031706444

Este *output* muestra que el estimado de bootstrap para $SE(\hat\alpha)$ es $0.0912$.

#### Estimando la Precisión de un Modelo de Regresión Lineal

El enfoque bootstrap puede utilizarse para evaluar la variabilidad de las estimaciones de los coeficientes y de las predicciones de un método de aprendizaje estadístico. Lo utilizaremos para evaluar la variabilidad de las estimaciones de $\beta_0$​ y $\beta_1$, el intercepto y la pendiente para el modelo de regresión lineal que utiliza `horsepower` para predecir `mpg` en el conjunto de datos `Auto`. Compararemos las estimaciones obtenidas mediante el bootstrap con aquellas obtenidas utilizando las fórmulas para $SE(\hat\beta_0)$ y $SE(\hat\beta_1)$.

Para utilizar la función `boot_SE()`, debemos escribir una función (su primer argumento) que tome un dataframe `D` y unos índices `idx` como sus únicos argumentos. Sin embargo, aquí queremos aplicar el bootstrap a un modelo de regresión específico, definido por una fórmula de modelo y datos. Mostramos cómo hacer esto en unos pocos pasos sencillos.

Comenzamos escribiendo una función genérica `boot_OLS()` para realizar el bootstrap de un modelo de regresión. Usamos la función `clone()` para hacer una copia de la fórmula que puede ser reajustada al nuevo `DataFrame`. Esto significa que cualquier característica derivada, como las definidas por `poly()` (que veremos a continuación), serán reajustadas en el `DataFrame` remuestreado.

In [66]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

Esto no es exactamente lo que se requiere como primer argumento para `boot_SE()`. Los dos primeros argumentos que especifican el modelo no cambiarán durante el proceso de bootstrap, y deseamos mantenerlos fijos. La función `partial()` del módulo functools realiza precisamente esto: toma una función como argumento y congela algunos de sus parámetros, comenzando desde el primero. La utilizamos para congelar los dos primeros argumentos de la fórmula del modelo en `boot_OLS()`.

In [72]:
hp_func  = partial(boot_OLS, MS(['horsepower']), 'mpg')

Al escribir `hp_func?` se mostrará que tiene dos argumentos, `D` e `idx`. Es una versión de `boot_OLS()` con los dos primeros argumentos congelados y, por lo tanto, es ideal como primer argumento para `boot_SE()`.

La función `hp_func()` puede ahora utilizarse para crear estimaciones bootstrap para los términos de intercepto y pendiente mediante el muestreo aleatorio con reemplazo de las observaciones. Primero demostramos su utilidad con 10 muestras bootstrap.

In [68]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
          rng.choice(Auto.index,
                     392,
                     replace=True)) for _ in range(10)])

array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

A continuación, utilizamos la función `boot_SE()` para calcular los errores estándar de $1000$ estimaciones bootstrap para los términos de intercepto y pendiente.

In [73]:
hp_se = boot_SE(hp_func,Auto,B=1000,seed=10)
hp_se

intercept     0.731542
horsepower    0.006095
dtype: float64

Esto indica que la estimación bootstrap para $SE(\hat\beta_0)$ es $0.73$ y que la estimación bootstrap para $SE(\hat\beta_1)$ es $0.0061$. Se pueden utilizar fórmulas estándar para calcular los errores estándar de los coeficientes de regresión en un modelo lineal. Estos se pueden obtener utilizando la función `summarize()` del módulo `ISLP.sm`.

In [74]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

Las estimaciones del error estándar para $\hat\beta_0$ y $\hat\beta_1$ obtenidas utilizando las "fórmulas" son $0.717$ para la intersección y $0.006$ para la pendiente. Curiosamente, estas difieren algo de las estimaciones obtenidas mediante el bootstrap. ¿Indica esto un problema con el bootstrap? De hecho, sugiere lo contrario. Recordemos que las fórmulas estándar dadas 

$$SE(\hat\beta_1) = \dfrac{\sigma^2}{\sum\limits_{i=1}^n(x_i-\bar{x})}\text{  y  }SE(\hat\beta_0)^2 = \sigma^2\left[\frac{1}{n} + \dfrac{\bar{x}^2}{\sum\limits_{i=1}^n(x_i-\bar{x})^2}\right]$$

se basan en ciertas suposiciones. Por ejemplo, dependen del parámetro desconocido $\sigma^2$, la varianza de $\varepsilon$. Luego estimamos $\sigma^2$ utilizando el $RSS$. Aunque la fórmula para los errores estándar no depende de que el modelo lineal sea correcto, la estimación de $\sigma^2$ sí lo hace. Por ejemplo, si existe una relación no lineal en los datos, los residuos de un ajuste lineal estarán inflados, al igual que $\sigma^2$. En segundo lugar, las fórmulas estándar asumen (poco realísticamente) que los $x_i$ son fijos y que toda la variabilidad proviene de la variación en los errores $\varepsilon$. El enfoque bootstrap no depende de ninguna de estas suposiciones, por lo que es probable que proporcione una estimación más precisa de los errores estándar de $\beta_0$ y $\beta_1$ que los resultados obtenidos con `sm.OLS`.